In [2]:
import arcgis # see if the install worked

In [3]:
import pandas as pd # us this to save the data as a CSV file on your Google Drive
from importlib import reload
import time
import math
import os
import re # us this for regular expressions
import glob
import csv # use this to write reviews as CSV later
from datetime import date

In [4]:
from arcgis.gis import GIS

In [5]:
from arcgis.geocoding import geocode
from arcgis.geocoding import analyze_geocode_input
arcgis.geocoding.get_geocoders(GIS())

[<Geocoder url:"https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer">]

In [6]:
dir_path = "/Users/willbpayne/Box/NJSOARH/Data/Picture of Subsidized Households/" # go to the folder with all the finished scraping
os.chdir(dir_path)
print(os.listdir(dir_path))

['COUNTY_2020_copy.xlsx', 'Data Comparisons', 'dictionary_2020.pdf', 'Mapping', 'NJ_County_2020.xlsx', 'NJ_PHA_2020_copy.xlsx', 'NJ_PLACE_2020_copy.xlsx', 'NJ_PROJECT_2020_copy.xlsx', 'NJ_PROJECT_2020_GEOCODE_TEST.csv', 'NJ_PROJECT_2020_GEOCODE_TEST.csv.sb-d0f96a0f-k2XIqX', 'NJ_PROJECT_2020_GEOCODE_TEST.xlsx', 'NJ_STATE_2020.xlsx', 'NJ_STATE_2020_copy.xlsx', 'PHA_2020_copy.xlsx', 'PHS 2020 Missing Tracts.xlsx', 'PLACE_2020_copy.xlsx', 'PROJECT_2020_copy.xlsx', 'PSH_2020_place.R', 'PSH_2020_project.R', 'PSH_2020_tract.R', 'Raw Data', 'STATE_2020_copy.xlsx', 'TRACT_MO_WY_2020_copy.xlsx', 'TRACT_NJ_2020_copy.xlsx']


In [7]:
with open("NJ_PROJECT_2020_GEOCODE_TEST.csv", 'r') as read_obj: # first load the file
    dict_reader = csv.DictReader(read_obj) # pass the file object to DictReader() to get the DictReader object
    dataframe = list(dict_reader)

In [22]:
addresstest = [dataframe[1]['name'],dataframe[1]['STD_ADDR'],dataframe[1]['STD_CITY'],dataframe[1]['STD_ZIP'],dataframe[1]['state']]
mysep = ", " 
geocodestring = mysep.join(addresstest)
geocodestring

'ATLANTIC CITY CONSUMER HOME, 45 N Kingston Ave, Atlantic City, 08401, NJ'

In [25]:
def geocode_project_locations(filepath): # function for geocoding all reviews from our scraped files
    
    dataframe_name = filepath.split('.')[0] # get the base name for the dataframe from the filepath

    with open(filepath, 'r') as read_obj: # first load the file
        dict_reader = csv.DictReader(read_obj) # pass the file object to DictReader() to get the DictReader object
        dataframe = list(dict_reader) # get a list of dictionaries from dct_reader
    
    dataframe_geocoded = [] # initialize an empty list to hold geocoded reviews

    for review in dataframe: # iterate through list
        
        review_geocoded = review
        addresstest = [review['name'],review['STD_ADDR'],review['STD_CITY'],review['STD_ZIP'],review['state']]
        mysep = ", " 
        geocodestring = mysep.join(addresstest)
        
        geocodeoutput = geocode(geocodestring)[0] # geocodes the user location and gives first choice
        print("Geocoded " + geocodestring) # print name of place geocoded
        review_geocoded['Lon'] = geocodeoutput['attributes']['X'] # saves Lon
        review_geocoded['Lat'] = geocodeoutput['attributes']['Y'] # saves Lat
        review_geocoded['Match_Score'] = geocodeoutput['score'] # saves score
        review_geocoded['Match_Type'] = geocodeoutput['attributes']['Type'] # saves score
        review_geocoded['Match_Address'] = geocodeoutput['attributes']['LongLabel'] # saves match address
        dataframe_geocoded.append(review_geocoded) # adds to new dictionary
        
    keys = dataframe_geocoded[0].keys() # get the list of columns to write

    output_filepath = dataframe_name + "_geocoded.csv" # new filepath in "Geocoded" folder with "_geocoded" appended
    
    with open(output_filepath, 'w')  as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(dataframe_geocoded)
    print("Wrote " + output_filepath + " to Google Drive!")

In [26]:
geocode_project_locations("NJ_PROJECT_2020_GEOCODE_TEST.csv")

Geocoded ASBURY PARK CONSUMER HOME, 1711 3rd Ave, Asbury Park, 07712, NJ
Geocoded ATLANTIC CITY CONSUMER HOME, 45 N Kingston Ave, Atlantic City, 08401, NJ
Geocoded ACACIA-LUMBERTON MANOR, 423 Landing St, Lumberton, 08048, NJ
Geocoded AL GOMER RESIDENCE, 219 S Orange Ave, South Orange, 07079, NJ
Geocoded ALEXANDRIA, 1 Alexandria Dr, Manalapan, 07726, NJ
Geocoded ALEXIAN MANOR, 122 7th St, Elizabeth, 07201, NJ
Geocoded ALLAIRE MANOR, 2011 New Bedford Rd, Wall Township, 07719, NJ
Geocoded ALPHA GROUP HOME, 429 Pohatcong St, Alpha, 08865, NJ
Geocoded AMITY VILLAGE #1, 325 Hawthorne Ave, Newark, 07112, NJ
Geocoded APARTMENT TWO, 18 4th Ave, Bridgewater, 08807, NJ
Geocoded ARCHITECTS HOUSING, 215 E Front St, Trenton, 08611, NJ
Geocoded ARLINGTON ARMS, 750 Grand St, Jersey City, 07304, NJ
Geocoded ARLINGTON HOUSE, 55-57 S Munn Ave, East Orange, 07018, NJ
Geocoded ASBURY TOWER, 1701 Ocean Ave, Asbury Park, 07712, NJ
Geocoded ASPEN HAMILTON APARTMENTS, 280 12th Ave, Paterson, 07514, NJ
Geocoded

Geocoded EATONTOWN CONSUMER HOME, 65 Broad St, Eatontown, 07724, NJ
Geocoded ECHELON TOWERS, 219 Laurel Rd, Voorhees, 08043, NJ
Geocoded EGG HARBOR CONSUMER HOME, , Egg Harbor City, 00, NJ
Geocoded ELEANOR LEVOVITZ SENIOR CITIZENS, 500 Clifton Ave, Lakewood, 08701, NJ
Geocoded ELIZABETH CENTER APARTMENTS, 809 Pearl St, Elizabeth, 07202, NJ
Geocoded ELLIOTT HOUSE FOR SENIORS INC., 710 N New York Ave, Atlantic City, 08401, NJ
Geocoded ELM COURT, 300 Elm Rd, Princeton, 08540, NJ
Geocoded B'NAI B'RITH ELMWOOD HOUSE INC, 444 Elmwood Rd N, Evesham, 08053, NJ
Geocoded ELSIE JAY I & II, 13 Newman St, Hackensack, 07601, NJ
Geocoded ELYSIAN ESTATES, 1300 Washington St, Hoboken, 07030, NJ
Geocoded ENGLEWOOD II, 68 William St, Englewood, 07631, NJ
Geocoded ESSEX PLAZA I, 1060 Broad St, Newark, 07102, NJ
Geocoded ESSEX PLAZA II, 15-31 Pennington St, Newark, 07102, NJ
Geocoded ESSEX PLAZA III, 150 Orchard St, Newark, 07102, NJ
Geocoded EVERGREEN MANOR, 50 Sade St, Clifton, 07013, NJ
Geocoded EVESHAM

Geocoded MIDDLE ROAD VILLAGE, 423 Middle Rd, Hazlet, 07730, NJ
Geocoded WATCHUNG TERRACE, 113 Mountain Ave, Middlesex, 08846, NJ
Geocoded MILFORD GROUP HOME, 1 John Lelo Ave, Milford, 08848, NJ
Geocoded MILL CREEK RESIDENCE, 191 Mill Creek Rd, Bayville, 08721, NJ
Geocoded MILL POND TOWERS ASSOCIATES, 45 Bassett Hwy, Dover, 07801, NJ
Geocoded MINNISINK VILLAGE, 101 Minnisink Dr, Matawan, 07747, NJ
Geocoded FIRST MONTCLAIR HOUSE, 56 Walnut St, Montclair, 07042, NJ
Geocoded SOUTH END GARDENS, 340 Orange Rd, Montclair, 07042, NJ
Geocoded MONTGOMERY GATEWAY EAST I, 336 Montgomery St, Jersey City, 07302, NJ
Geocoded MOUNT CARMEL TOWERS, 268 Oakwood Ave, Orange, 07050, NJ
Geocoded MULLEN MANOR, 300 Hickstown Rd, Sicklerville, 08081, NJ
Geocoded HODSON MANOR, 20 S Main St, Glassboro, 08028, NJ
Geocoded MOUNT CALVARY II, 100 Chadwick Ave, Newark, 07108, NJ
Geocoded MOUNT CALVARY I, 244 Chadwick Ave, Newark, 07108, NJ
Geocoded MUCH DIGNITY HOUSE, 18 Cedar Ter, Randolph, 07869, NJ
Geocoded MUHLEN

Geocoded RIVERVIEW TOWERS II, 125 Presidential Blvd, Paterson, 07522, NJ
Geocoded ROHRER TOWERS II, 300 W Crystal Lake Ave, Haddonfield, 08033, NJ
Geocoded RIVER RUN APARTMENTS/ROOSE, 100 Roosevelt Ave, Carteret, 07008, NJ
Geocoded ROOSEVELT SENIOR CITIZENS HOUS, 9 N Valley Rd, Roosevelt, 08555, NJ
Geocoded ROOSEVELT VILLAGE, 100 Roosevelt Ave, Carteret, 07008, NJ
Geocoded RUTHERFORD SENIOR HOUSING, 67 Kip Ave, Rutherford, 07070, NJ
Geocoded RIDGEFIELD PARK, 278 Main St, Ridgefield Park, 07660, NJ
Geocoded SALEM TOWERS APARTMENTS - 5600, 98 High St, Orange, 07050, NJ
Geocoded SALEM TOWERS APARTMENTS, 98 High St, Orange, 07050, NJ
Geocoded SALEM TOWERS APARTMENTS, 98 High St, Orange, 07050, NJ
Geocoded SEBASTIAN VILLA APARTMENTS - 5621, 2305 W Bangs Ave, Neptune, 07753, NJ
Geocoded LIBERTY APARTMENTS, 1519 Baltic Ave, Atlantic City, 08401, NJ
Geocoded NORA GARDENS, 1775 Burnet Ave, Union, 07083, NJ
Geocoded WASHINGTON STREET APARTMENTS, 440 Washington St, Newark, 07102, NJ
Geocoded ST. 

Geocoded FREEMAN APARTMENTS, 41 Hillside Ave, Springfield, 07081, NJ
Geocoded PROJECT LIVE VI, INC., 30 Rutgers St, Maplewood, 07040, NJ
Geocoded PROJECT LIVE VII, INC., 255 Gardner Ave, South Orange, 07079, NJ
Geocoded PASSAIC CONSUMER HOME, 234 Paulison Ave, Passaic, 07055, NJ
Geocoded HALEDON CONSUMER HOME, 639 Belmont Ave, Haledon, 07508, NJ
Geocoded ACCESS I GROUP HOME, 10 Entin Rd, Parsippany, 07054, NJ
Geocoded NEWMAN SPRINGS, 11 Williams Rd, Tinton Falls, 07753, NJ
Geocoded PLAINFIELD AVE., INC., 337 Plainfield Ave, Edison, 08817, NJ
Geocoded ABSECON CONSUMER HOME, 303 Crestview Ave, Galloway, 08205, NJ
Geocoded PROJECT LIVE IX, INC., 7 Walnut St, Livingston, 07039, NJ
Geocoded CLIFTON CONSUMER HOME, 123 Luisser St, Clifton, 07012, NJ
Geocoded WHEATON POINTE AT EAST WINDSOR, 20 Lanning Blvd, East Windsor, 08520, NJ
Geocoded LUTHERAN SENIOR RESIDENCE, 5610 S Crescent Blvd, Pennsauken, 08109, NJ
Geocoded B'NAI B'RITH CHESILHURST HOUSE, 227 White Horse Pike, Chesilhurst, 08089, NJ

Geocoded VILLAGE AT ST PETERS, 25 W Black Horse Pike, Pleasantville, 08232, NJ
Geocoded MANCHESTER PINES, , Manchester Twp, 08759, NJ
Geocoded CARING SENIOR HOUSING II, 415 W Delilah Rd, Pleasantville, 08232, NJ
Geocoded CPNJ WARREN RESIDENCE INC, 115 Stirling Rd, Warren, 07059, NJ
Geocoded CSPNJ HOMES 2007, 4800 White Horse Pike, Egg Harbor City, 08215, NJ
Geocoded CARING HOMES 2007 - ATLANTIC COUNTY, 5905 Buttercup Ln, Mays Landing, 08330, NJ
Geocoded CARING HOMES 2007 - RIDGE AVENUE, 5053 Ridge Ave, Egg Harbor Twp, 08234, NJ
Geocoded ATLANTIC COUNTY HOMES INC., 527 S 10th Ave, Galloway, 08205, NJ
Geocoded LINDENWOLD GARDENS, 350 Blackwood Clementon Rd, Pine Hill, 08021, NJ
Geocoded PINE HILL GARDENS, 350 Blackwood Clementon Rd, Pine Hill, 08021, NJ
Geocoded BENEDICTS PLACE, 206 Saint Marys Dr, Cherry Hill, 08003, NJ
Geocoded THE APARTMENTS AT ST ELIZABETH, 176 Hussa St, Linden, 07036, NJ
Geocoded CPNJ PLAINFIELD RESIDENCE II INC, 1019 Hillside Ave, Plainfield, 07060, NJ
Geocoded CPN

Geocoded Woodrow Wilson Homes III, , Long Branch, 07740, NJ
Geocoded MARION GRDNS, 57 Dales Ave, Jersey City, 07306, NJ
Geocoded BOOKER T WASHINGTON APTS, 90 Fremont St, Jersey City, 07302, NJ
Geocoded HUDSON GRDNS, 27 Palisade Ave, Jersey City, 07306, NJ
Geocoded HOLLAND GRDNS, 235 16th St, Jersey City, 07310, NJ
Geocoded CURRIES WOODS, 3 New Heckman Dr Apt 861, Jersey City, 07305, NJ
Geocoded BERRY GRDNS, 199 Ocean Ave, Jersey City, 07305, NJ
Geocoded DWIGHT STREET HOMES, 125A Dwight St, Jersey City, 07305, NJ
Geocoded LAFAYETTE II, 110 Manning Ave, Jersey City, 07304, NJ
Geocoded Lafayette Senior Living Center, 463 Pacific Ave Apt 311, Jersey City, 07304, NJ
Geocoded Pacific Court, 2A Cannon Dr, Jersey City, 07304, NJ
Geocoded WOODWARD TERRACE, 3771 Van Horne St, Jersey City, 07304, NJ
Geocoded THOMAS STEWART APTS, 88 Erie St, Jersey City, 07302, NJ
Geocoded GLORIA ROBINSON COURT HOMES I, 8 Harvey Ave, Jersey City, 07306, NJ
Geocoded BARBARA PLACE TERRACE, 3621 Van Horne St, Jersey 

Geocoded PARKVIEW & D'ORAZIO TERRACE, 84 S West Ave Apt 70, Vineland, 08360, NJ
Geocoded VINELAND HA, 64 Arcadia Pl, Vineland, 08360, NJ
Geocoded WM G ROHRER TWS, 25 Wynnewood Ave, Westmont, 08108, NJ
Geocoded ANN J FERGUSON TOWERS, 1601 Dill Ave, Linden, 07036, NJ
Geocoded SENIOR CITZ BLDG, 215 E Blackwell St, Dover, 07801, NJ
Geocoded MONMOUTH CT, 107 Throckmorton St, Freehold, 07728, NJ
Geocoded WOOSTER TOWERS, 22 Gibbsboro Rd, Clementon, 08021, NJ
Geocoded PENN TWS SOUTH, 40 S Broad St, Penns Grove, 08069, NJ
Geocoded LIBERTY TWS, 32 Liberty St, Newton, 07860, NJ
Geocoded COLLINGSWOOD ARMS, 30 Washington Ave, Collingswood, 08108, NJ
Geocoded COMMISSIONERS CT APTS, 3700 New Jersey Ave Unit A1, Wildwood, 08260, NJ
Geocoded SANDMAN TWS, 3700 New Jersey Ave, Wildwood, 08260, NJ
Geocoded Morris County Housing, 221 Mount Pleasant Ave, Dover, 07801, NJ
Geocoded 204-1 scattered sites, 1 Walnut Ave, Pitman, 08071, NJ
Geocoded CARINO PARK, 100 Chestnut St, Williamstown, 08094, NJ
Geocoded DE